In [1]:
from joblib import dump, load
import os
import string
import pickle

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

import statsmodels.api as sm

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import plot_confusion_matrix
from sklearn.linear_model import Ridge, GammaRegressor

# PyTorch stuff
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
import torch.nn.functional as F

# Autoselect target device.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device {device}")

from timeit import default_timer as timer
from datetime import timedelta

os.chdir('/home/lawrence/Personal/Masters/COMP0087_ Natural_Language_Processing/Project/nlp-group-project-main/nvdm')
from models import nvdm

Using device cuda


# Helper Functions
## Text Processor

In [2]:
# Instance of Lemmatizer
LEMMATIZER = WordNetLemmatizer()
STOPWORDS = stopwords.words('english')

def preprocess_text(text):
    """ Process a single line of text. """

    # Strip trailing characters if any (e.g. newline)
    text_new = text.strip()
    
    # Remove puncuation
    text_new = ''.join(ch for ch in text_new if ch not in string.punctuation)

    # Lower case
    text_new = text_new.lower()
    
    # Tokenise by space
    tokens = text_new.split()
    
    # Remove stopwords
    tokens = [word for word in tokens if word not in STOPWORDS]

    # Lemmatise each word
    tokens = [LEMMATIZER.lemmatize(word) for word in tokens]
    
    text_new = ' '.join(tokens)

    return text_new

## Model Wrapper

In [3]:
class Model:
    """ Base class for clustering models.
    
    Basically a wrapper for a variety of models.
    """
    def __init__(self):
        pass
    
    def fit(self, X):
        """ Train the model. """
        raise NotImplementedError()
    
    def transform(self, X):
        """ Apply model to new data.
        
        Should output a topic-document matrix,
        where each element is a score indicating how likely the document
        should be assigned to the topic.
        For sklearn LDA, transform() does this by default.
        """
        raise NotImplementedError()

    @property
    def topic_vocab_matrix(self):
        """ Each model should be able to return a topic-vocab matrix
        containing a score (e.g. probability) of a word in the vocabulary
        occuring in the k^th topic. """
        pass
    
    # TODO: topic-document matrix
    # TODO: perplexity? Don't think K-Means has a notion of perplexity
    # (because we need probabilities).

    
class KMeansModel(Model):
    """ Wrapper for scikit-learn KMeans. """
    def __init__(self, num_topics):
        self.model = KMeans(
            n_clusters = num_topics,
            init='k-means++',
            max_iter = 300,
            n_init = 10,
            verbose = False)
    
    def fit(self, X):
        self.model.fit(X)

    def transform(self, X):
        """ Returns a topic-document matrix of distances per cluster. """
        return -self.model.transform(X)
    
    @property
    def topic_vocab_matrix(self):
        """ Return K-Means clusters.
        
        ndarray of shape (num_topics, n_features)
        """
        return self.model.cluster_centers_


class LDAModel(Model):
    """ Wrapper for scikit-learn LDA. """
    def __init__(self, num_topics):
        self.model = LatentDirichletAllocation(
            n_components=num_topics,
            max_iter=5,
            learning_method='online',
            learning_offset=50.,
            n_jobs=-1)
    
    def fit(self, X):
        self.model.fit(X)
    
    def transform(self, X):
        """ Returns a topic-document matrix of probabilities. """
        return self.model.transform(X) 
    
    @property
    def topic_vocab_matrix(self):
        """ Gets the components_ attribute of LDA, normalized
        
        Quoting sklearn docs:
        Variational parameters for topic word distribution.
        Since the complete conditional for topic word distribution is a Dirichlet,
        components_[i, j] can be viewed as pseudocount that represents
        the number of times word j was assigned to topic i.
        It can also be viewed as distribution over the words for each topic after normalization:
        model.components_ / model.components_.sum(axis=1)[:, np.newaxis].
        """
        # return self.model.components_
        return self.model.components_ / self.model.components_.sum(axis=1)[:, np.newaxis]

class NVDMModel(Model):
    """ PyTorch NVDM model.
    
    Loads a pretrained model from disk.
    """
    def __init__(self, model_path, vocab_size, hidden_size=500, num_topics=300):
        self.model = NVDM(vocab_size, hidden_size, num_topics, 1, "cpu")
        self.model.load_state_dict(torch.load(model_path, map_location="cpu"))
        self.model.eval()
        
        decoder = self.model.decoder[0]
        weights = decoder.weight.data.detach().clone().cpu().numpy()
        self.topic_vocab = weights.T
    
    def fit(self, X):
        """ We don't train the model here because it takes too long. """
        pass
    
    def transform(self, X):
        """ Output a topic-document matrix. """
        n_doc, n_vocab = X.shape
        n_topic = self.topic_vocab.shape[0]
        
        # shape (n_doc, n_topic)
        # Score of each document for a topic is the average scores
        # of the document's words in the topic.
        topic_doc = X @ self.topic_vocab.T
        
        # Optionally, normalize by document length.
        topic_doc = topic_doc / X.sum(axis=1, keepdims=True)
        
        return topic_doc
    
    @property
    def topic_vocab_matrix(self):
        """ Returns the learned semantic embeddings of each word. """
        return self.topic_vocab

## Evaluation

In [4]:
# Topic coherence.
def umass_score(tf):
    """ Compute topic coherence using UMass metric.
    
    Ref: http://qpleple.com/topic-coherence-to-evaluate-topic-models/
    
    tf: term-frequency matrix for each document.
        Each i^th row is the BOW representation of the i^th document.
    """
    
    # D(wi): count of documents containing the word wi (i.e. df)
    Dwi = np.array(np.sum(tf > 0, axis=0))[0]

    W_bin = np.zeros_like(tf)
    W_bin[tf > 0] = 1
    
    # D(wi, wj): count of documents containing both words wi and wj
    Dwi_wj = W_bin.T @ W_bin

    score_umass = np.log((Dwi_wj + 1)/ Dwi)
    
    return score_umass

def topic_coherence(topic_vocab, n_top_words, pair_score):
    """ Compute the topic coherence of each topic,
    given a learned topic-vocabulary matrix, the number of top words to use
    and a matrix of pairwise scores (e.g. umass_score output)
    
    topic_vocab: dimensions (number of topics, vocabulary size).
    model.components_ for LDA, and the "semantic embedding" matrix in the decoder for NVDM.
    
    pair_score: matrix of scores (e.g. UMass)
    """
    coherences = []
    for topic_idx, topic in enumerate(topic_vocab):
        top_features_ind = topic.argsort()[:-n_top_words - 1:-1]
        coh = 0
        for i in range(len(top_features_ind)):
            for j in range(i):
                coh += pair_score[top_features_ind[i], top_features_ind[j]]
        coherences.append(coh)
    return coherences

def plot_top_words(topic_vocab, feature_names, n_top_words, title):
    """ Given a topic-vocabulary matrix containing scores
    (e.g. probabilities, higher the better),
    plot the top words as a frequency bar-graph for each topic.
    
    e.g. set topic_vocab=model._components for LDA.
    """
    K = len(topic_vocab)
    n_x = 5
    n_y = int(np.ceil(K / n_x))
    fig, axes = plt.subplots(n_y, n_x, figsize=(2.5 * n_x, 4 * n_y), sharex=True)
    axes = axes.flatten()
    for topic_idx, topic in enumerate(topic_vocab):
        top_features_ind = topic.argsort()[:-n_top_words - 1:-1]
        top_features = [feature_names[i] for i in top_features_ind]
        weights = topic[top_features_ind]

        ax = axes[topic_idx]
        ax.barh(top_features, weights, height=0.7)
        ax.set_title(f'Topic {topic_idx +1}',
                     fontdict={'fontsize': 14})
        ax.invert_yaxis()
        ax.tick_params(axis='both', which='major', labelsize=12)
        for i in 'top right left'.split():
            ax.spines[i].set_visible(False)
        fig.suptitle(title, fontsize=20)

    # plt.subplots_adjust(top=0.90, bottom=0.05, wspace=0.90, hspace=0.3)
    
    fig.tight_layout()
    plt.show()
    
    
def daily_adjusted_r2(returns, features):
    """
    Takes in a dates by stocks matrix (T x M) of stock returns and a stocks by features matrix (M x K) 
    of features.
    
    Loops over dates. On each date, performs a OLS regression of M stock returns against a M x (K + 1) matrix
    of features, including an intercept which has been added. Then records the adjusted R2 of that regression
    on that date.
    
    Outputs a dataframe of length T of R2 values.
    """
    all_dates = returns.index
    adj_r2 = []

    for dd in all_dates:
        # removing features we don't have returns for
        reg_data = returns.loc[[dd]].transpose().join(features).dropna(axis=0).values
        y = reg_data[:, 0]
        X = reg_data[:, 1:]

        std_scaler = StandardScaler()
        X = std_scaler.fit_transform(X)

        X = sm.add_constant(X, prepend=False)
        ols_model = sm.OLS(y, X)
        res = ols_model.fit()
        adj_r2.append(res.rsquared_adj)
    return pd.DataFrame(index=all_dates, data=adj_r2)


def cv_score_ridge(dependent, features):
    trials = 200
    alphas = [0.01, 0.1, 1, 10, 100, 200, 500, 1000]
    best_score = 0
    stdev_best = 0
    alpha_best = 0
    for alpha in alphas:
        res_alpha = []
        for _ in range(trials):
            X_train, X_test, y_train, y_test = train_test_split(features, dependent)
            my_regression = Ridge(alpha=alpha)
            std_scaler = StandardScaler()
            X_train = std_scaler.fit_transform(X_train)
            X_test = std_scaler.transform(X_test)
            my_regression.fit(X_train, y_train)
            res_alpha.append(my_regression.score(X_test, y_test))
        if np.mean(res_alpha) > best_score:
            best_score = np.mean(res_alpha)
            stdev_best = np.std(res_alpha)
            alpha_best = alpha
    return best_score, stdev_best, alpha_best


def cv_score_gamma(dependent, features):
    trials = 200
    alphas = [0.01, 0.1, 1, 10, 100, 200, 500, 1000]
    best_score = 0
    stdev_best = 0
    alpha_best = 0
    for alpha in alphas:
        res_alpha = []
        for _ in range(trials):
            X_train, X_test, y_train, y_test = train_test_split(features, dependent)
            my_regression = GammaRegressor(alpha=alpha)
            std_scaler = StandardScaler()
            X_train = std_scaler.fit_transform(X_train)
            X_test = std_scaler.transform(X_test)
            my_regression.fit(X_train, y_train)
            res_alpha.append(my_regression.score(X_test, y_test))
        if np.mean(res_alpha) > best_score:
            best_score = np.mean(res_alpha)
            stdev_best = np.std(res_alpha)
            alpha_best = alpha
    return best_score, stdev_best, alpha_best  

## Data Loaders

In [5]:
def load_sp500(path, preprocess=False):
    """ Load S&P500 data from the per-company text files in the supplied directory path.
    
    Within the directory, each file is named "<ticker>_<sector>.txt".
    Each contains the business description (BD) of the company.
    
    If preprocess is True, the preprocess the business descriptions at the same time.
    """
    filenames = os.listdir(path)

    tickers = []
    sectors = []
    bds = []
    for fn in filenames:
        prefix = fn.split('.txt')[0]
        ticker, sector = prefix.split('_')
        filepath = os.path.join(path, fn)
        with open(filepath, 'r', encoding="utf8") as f:
            bd = f.read().strip()
        
        if preprocess:
            bd = preprocess_text(bd)

        tickers.append(ticker)
        sectors.append(sector)
        bds.append(bd)
    
    return tickers, sectors, bds


def load_bds1(path, preprocess=False, exclude_tickers=None):
    """ Load data from the business data, given the file path (e.g. "data/bds_1.txt").
    
    In the file, each company has two consecutive lines.
    The first line is <company ticker>:<CIK> (we only care about the ticker)
    and the second line is the company business description.
    
    exclude_tickers is a list of tickers that we want to ignore in bds_1.txt.
    For example, we can use this to exclude any S&P500 companies to avoid
    overlapping of datasets.
    """
    
    with open(path, "r", encoding="utf8") as f:
        lines = f.readlines()

    company_ids_all = [ln.strip() for ln in lines[0::2]]
    company_descriptions_all = [ln.strip() for ln in lines[1::2]]
    company_tickers = [x.split(':')[0] for x in company_ids_all]

    exclusion_set = set(exclude_tickers) if exclude_tickers is not None else set()

    tickers = []
    bds = []
    
    # Some business descriptions are too short (or even empty),
    # so we only keep those with a length (number of characters) deemed reasonable.
    bd_valid_length = 3000
    for ticker, bd in zip(company_tickers, company_descriptions_all):
        if ticker not in exclusion_set and len(bd) >= bd_valid_length:
            tickers.append(ticker)
            
            if preprocess:
                bd = preprocess_text(bd)
            bds.append(bd)
    
    return tickers, bds


def load_returns(path, tickers, start_d=np.datetime64('2018-01-01'), end_d=np.datetime64('2020-01-01')):
    """
    Loads price data from path, between a set of dates and converts to daily log returns. 
    
    Data only available for S&P 500 stocks that were in index over past 20 years. Supply list of tickers to 
    subset this.
    
    Example path:
    '/home/lawrence/Personal/Masters/COMP0087_ Natural_Language_Processing/Project/Data/MarketData/Price.csv'
    
    Please install Pandas library to get working (pip install pandas)
    """
    
    # loads    
    price_data = pd.read_csv(path)
    
    # subset desired tickers from price file
    select_these = np.in1d(price_data.tic.values, list(tickers))
    
    # price file stored columns wise. Select columns we want.
    price_sp50 = price_data.loc[select_these, ['tic', 'datadate', 'prccd']]
    
    # format dates to use pandas date handling
    price_sp50['datadate'] = pd.to_datetime(price_sp50['datadate'], format='%Y%m%d')
    
    # pivot to dates by stocks table
    price_sp50 = pd.pivot_table(price_sp50,index='datadate',columns='tic',values='prccd')
    
    # fill in missing prices so we can calculate returns over holidays
    price_sp50 = price_sp50.ffill(limit=5)
    
    # subset business days only
    business_ds = pd.date_range(start_d, end_d, freq='B')
    price_sp50 = price_sp50.reindex(business_ds)
    price_sp50 = price_sp50.loc[~np.all(np.isnan(price_sp50.values), axis=1), :]
    
    # remove stocks with missing data
    price_sp50 = price_sp50.dropna(axis=1)
    
    # we now have a continuous price series for all stocks. We convert these into changes in price, 
    # i.e. log returns. Logs used as dampens outliers for subsequent OLS regression.
    returns_sp50 = np.log(price_sp50) - np.log(price_sp50.shift(1))
    return returns_sp50.dropna(axis=0)

# Load Data

In [6]:
sp500_path = '/home/lawrence/Personal/Masters/COMP0087_ Natural_Language_Processing/Project/Data/SP500'
sp500_tickers, sp500_sectors, sp500_bds = load_sp500(sp500_path, preprocess=True)

bds1_path = '/home/lawrence/Personal/Masters/COMP0087_ Natural_Language_Processing/Project/Data/bds_1.txt'
bds1_tickers, bds1_bds = load_bds1(bds1_path, preprocess=True, exclude_tickers=sp500_tickers)

stock_returns_path = '/home/lawrence/Personal/Masters/COMP0087_ Natural_Language_Processing/Project/Data/MarketData/Price.csv'
sp500_returns = load_returns(stock_returns_path, sp500_tickers)

/home/lawrence/miniconda3/envs/comp0086_Exercises/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3337: DtypeWarning: Columns (1,4) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [7]:
total_returns = pd.DataFrame(sp500_returns.mean(axis=0))
total_returns.columns = ['returns']
total_stdev = pd.DataFrame(sp500_returns.std(axis=0))
total_stdev.columns = ['stdev']

# Extract Text Features

In [8]:
n_features = 4000
tf_vectorizer = CountVectorizer(max_features=n_features, max_df=0.95, min_df=2)
tf_vectorizer.fit(bds1_bds)

CountVectorizer(max_df=0.95, max_features=4000, min_df=2)

In [9]:
X_sp500 = tf_vectorizer.transform(sp500_bds).toarray()
X_bds = tf_vectorizer.transform(bds1_bds).toarray()

um_score_bds = umass_score(X_bds)

# Setting up NVDM Model

In [10]:
class BDDataset(Dataset):
    """ Very simple dataset object. Stores all the passages.
    
    This is just for compatibility with PyTorch DataLoader.
    """
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]

In [11]:
dataset = BDDataset(torch.tensor(X_sp500, dtype=torch.float32))
batch_size = 64
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
hidden_size = 250

In [12]:
# training loop
def train(model, data_loader, outer_epochs=1000, print_every=100, device="cpu"):

    # Trains both the encoder and decoder at the same time.
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    start_time = timer()
    print("Start training...")


    for epoch in range(outer_epochs):

        loss_sum = 0.0
        rec_sum = 0.0
        kl_sum = 0.0
        n = len(data_loader)

        for text in data_loader:
            text = text.to(device)

            optimizer.zero_grad()
            loss_dict = model(text, kl_weight=1.0)
            loss = loss_dict["total"].sum()
            loss.backward()

            optimizer.step()

            # For printing
            loss_sum += loss.item()
            rec_sum += loss_dict["rec"].sum().item()
            kl_sum += loss_dict["kl"].sum().item()

        if (epoch + 1) % print_every == 0:
            print(f"[Time: {timedelta(seconds=timer() - start_time)}, Epoch {epoch + 1}] Loss {loss_sum/n}, Rec {rec_sum/n}, KL {kl_sum/n}")

# K Means

In [14]:
os.chdir('/home/lawrence/Personal/Masters/COMP0087_ Natural_Language_Processing/Project/nlp-group-project-main/Models/')

In [18]:
k = 20
kmean = KMeansModel(k)
kmean.fit(X_bds)
p_file = open(f'KMeans_{k}', 'wb')
pickle.dump(kmean.model, p_file)
p_file.close()

# LDA

In [19]:
k = 20
lda = LDAModel(k)
lda.fit(X_bds)
p_file = open(f'LDA_{k}', 'wb')
pickle.dump(lda.model, p_file)
p_file.close()

# NVDM

In [17]:
k = 20
hidden_size = 250
epochs = 100
model_nvdm = nvdm.NVDM(n_features, hidden_size, k, 1, device)
model_nvdm = model_nvdm.to(device)
model_nvdm.train()
train(model_nvdm, data_loader, outer_epochs=epochs, print_every=500, device=device)
torch.save(model_nvdm, f'NVDM_{k}')

Start training...
